# train_party 데이터와 train_label 데이터 병합
# 순서
## 1. [Importing Packages 및 Loading Data](#1)
## 2. [파티 멤버 각각을 행으로 분리](#2)
## 3. [id별로 파티 시간 합치기 - groupby](#3)
## 4. [train_label 데이터와 병합 - merge](#4)

## 1. Importing Packages 및 Loading Data<a name = 1></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
path = 'data/final_data_rev/train/'
DF_temp = pd.read_csv(path+'party_temp2.csv')
DF_label = pd.read_csv(path+'train_label.csv')

In [3]:
DF_temp.head()

,party_start_week,party_end_week,hashed,minutes
0,1,1,11fc85879e5ac9d5c83bfba10d73c4c84c154b9f4d9e1d...,26.527367
1,3,3,7176c1516207692857535c30a4650b8e8e586af1fed0fd...,122.622317
2,3,3,8092e194a750aae539862ed4405f67a6dd5b492e7e57e3...,10.250083
3,4,4,4ec597c569b92bd0e1bae4e2a06e13b9657fb81795e194...,25.637600
4,4,4,a4b6aea6cb58e43911e7cb7d6c0497197db7c4ed16e1c9...,21.563700


## 2. 파티 멤버 각각을 행으로 분리<a name=2></a>
아래 함수는 github의 재야의 숨은 고수가 만들어놓은 함수입니다. 코드를 이해하려고 노력했으나, 쉽지 않네요.<br/>
혹시나 아래 함수 코드를 이해하신 분은 저에게도 설명을 해주시면 감사하겠습니다!!

In [4]:
def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

In [5]:
start = datetime.now()
new_df = splitDataFrameList(DF_temp, 'hashed', ',')
print("run time : ",datetime.now()-start)
print("Data shape : ",new_df.shape)
new_df.head(10)

run time :  0:27:38.259916
Data shape :  (34284282, 4)


,hashed,minutes,party_end_week,party_start_week
0,11fc85879e5ac9d5c83bfba10d73c4c84c154b9f4d9e1d...,26.527367,1,1
1,cbe1b3a37225ef716a739d504fc13e45466487976546d5...,26.527367,1,1
2,9537dfd85f3afe656d736e9950118548d493b05f66c524...,26.527367,1,1
3,b08d19ef636db20720d8456ba6457a215a35dae9450e82...,26.527367,1,1
4,114c3f7736f37824cdacdac7f288054b21c526470c2ea7...,26.527367,1,1
5,831fc4830fc0da35a5c505e5ae7665f0c1cfa5b3df28e9...,26.527367,1,1
6,7176c1516207692857535c30a4650b8e8e586af1fed0fd...,122.622317,3,3
7,2209514bcb9b8408d678cb6e7393eafec86bea2531e4eb...,122.622317,3,3
8,fa1958e7846690088273f7f55c04bdbee9715ff2d3c085...,122.622317,3,3
9,e2f97c5cb6a71a627eb4590856d47bfb0f80c68ca76429...,122.622317,3,3


파티 구성원을 1열로 나열하니 34,559,610개나 되네요 ...<br/>
datetime.now()를 사용해서 코드의 run time을 확인할 수 있습니다.<br/>
혹시 모르니깐 결과가 맞는지 맨 마지막 2개 파티로 확인해보겠습니다.

In [6]:
display(DF_temp.tail(2))
display(DF_temp.iloc[-1,0])
display(DF_temp.iloc[-2,0])
new_df.tail(9)

,party_start_week,party_end_week,hashed,minutes
6962339,7,7,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...,3.96700
6962340,6,6,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...,3.50345


6

7

,hashed,minutes,party_end_week,party_start_week
34284273,8657cb79d60cf11c28a786b6ff515d3eaf1a16ff7279a3...,0.45840,7,7
34284274,02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...,3.96700,7,7
34284275,769f00750d87fa59f8b6083aae62fb394e165101796fe5...,3.96700,7,7
34284276,3893d4da8854ada5484099d8017c1504d62d98cf11fb2a...,3.96700,7,7
34284277,039f09f2d5c1206d51c9299a774fe2041db91dcf60b487...,3.96700,7,7
34284278,2a8053c8632f8116fa8a8319709d73c09f3cf409d56afb...,3.96700,7,7
34284279,967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...,3.50345,6,6
34284280,a0b6ecbec654b18fe36ebe6230e25a653fb12125733583...,3.50345,6,6
34284281,3193ab18168bcadbcb8342c06c4a35fa0d6e58d9619fe8...,3.50345,6,6


아주 잘 맞습니다! 다음 단계로 넘어갑시다!

In [7]:
### 2차 전처리 추가 코드입니다.(2018.08.03)
week = 1

DF_week = new_df[new_df['party_start_week'] == week]
DF_week_groupby = DF_week.groupby('hashed').agg('sum')
DF_week_groupby = DF_week_groupby.reset_index()

DF_week_groupby = DF_week_groupby.drop(['party_end_week', 'party_start_week'], axis = 1)
DF_week_groupby.columns = ['acc_id', str(week)+'_party_time']

DF_merge = pd.merge(left = DF_label, right = DF_week_groupby, how='left', on='acc_id')

,hashed,minutes,party_end_week,party_start_week
0,000020b4fa2af1e40c813436e5054bce70b703d0039dcd...,4104.470133,50,48
1,000291b3c6e9c6012afed9dd99bbc834082622667fe3b7...,8369.737217,12,11
2,0002cb165b57f8ab2bea2e387a3bebcd1ee57b44229150...,5177.057167,55,54
3,00036e5b6a197c196fa10fc0ad4e2853b22294dff64d27...,159.565000,14,14
4,00038d64fb50e6967fa4488c077614ba9509ed0869f0ad...,3334.939967,2,2
5,0003b127aa1c0c34490db8817574482464aad9f99cffde...,174.979467,7,7
6,000572abb83d1eb94235085bc9400a8ead0afb42d5e952...,12651.321067,4,2
7,00062d723bff3fcefb50370a299af303ecbadf6dfba59b...,302.640283,15,15
8,0006502148dc2533ef4ac4b9939ee19f18483a7c3c7256...,321.728967,143,143
9,0008f58eaf2c598838dd8cf5e3ab598025f1381abdfdd7...,338.517033,14,14


In [27]:
### 2차 전처리 추가 코드입니다.(2018.08.03)
week = 2

DF_week = new_df[new_df['party_start_week'] == week]
DF_week_groupby = DF_week.groupby('hashed').agg('sum')
DF_week_groupby = DF_week_groupby.reset_index()

DF_week_groupby = DF_week_groupby.drop(['party_end_week', 'party_start_week'], axis = 1)
DF_week_groupby.columns = ['acc_id', str(week)+'_party_time']

DF_merge = pd.merge(left = DF_merge, right = DF_week_groupby, how='left', on='acc_id')

In [28]:
### 2차 전처리 추가 코드입니다.(2018.08.03)
week = 3

DF_week = new_df[new_df['party_start_week'] == week]
DF_week_groupby = DF_week.groupby('hashed').agg('sum')
DF_week_groupby = DF_week_groupby.reset_index()

DF_week_groupby = DF_week_groupby.drop(['party_end_week', 'party_start_week'], axis = 1)
DF_week_groupby.columns = ['acc_id', str(week)+'_party_time']

DF_merge = pd.merge(left = DF_merge, right = DF_week_groupby, how='left', on='acc_id')

In [29]:
### 2차 전처리 추가 코드입니다.(2018.08.03)
week = 4

DF_week = new_df[new_df['party_start_week'] == week]
DF_week_groupby = DF_week.groupby('hashed').agg('sum')
DF_week_groupby = DF_week_groupby.reset_index()

DF_week_groupby = DF_week_groupby.drop(['party_end_week', 'party_start_week'], axis = 1)
DF_week_groupby.columns = ['acc_id', str(week)+'_party_time']

DF_merge = pd.merge(left = DF_merge, right = DF_week_groupby, how='left', on='acc_id')

In [30]:
### 2차 전처리 추가 코드입니다.(2018.08.03)
week = 5

DF_week = new_df[new_df['party_start_week'] == week]
DF_week_groupby = DF_week.groupby('hashed').agg('sum')
DF_week_groupby = DF_week_groupby.reset_index()

DF_week_groupby = DF_week_groupby.drop(['party_end_week', 'party_start_week'], axis = 1)
DF_week_groupby.columns = ['acc_id', str(week)+'_party_time']

DF_merge = pd.merge(left = DF_merge, right = DF_week_groupby, how='left', on='acc_id')

In [31]:
### 2차 전처리 추가 코드입니다.(2018.08.03)
week = 6

DF_week = new_df[new_df['party_start_week'] == week]
DF_week_groupby = DF_week.groupby('hashed').agg('sum')
DF_week_groupby = DF_week_groupby.reset_index()

DF_week_groupby = DF_week_groupby.drop(['party_end_week', 'party_start_week'], axis = 1)
DF_week_groupby.columns = ['acc_id', str(week)+'_party_time']

DF_merge = pd.merge(left = DF_merge, right = DF_week_groupby, how='left', on='acc_id')

In [32]:
### 2차 전처리 추가 코드입니다.(2018.08.03)
week = 7

DF_week = new_df[new_df['party_start_week'] == week]
DF_week_groupby = DF_week.groupby('hashed').agg('sum')
DF_week_groupby = DF_week_groupby.reset_index()

DF_week_groupby = DF_week_groupby.drop(['party_end_week', 'party_start_week'], axis = 1)
DF_week_groupby.columns = ['acc_id', str(week)+'_party_time']

DF_merge = pd.merge(left = DF_merge, right = DF_week_groupby, how='left', on='acc_id')

In [33]:
### 2차 전처리 추가 코드입니다.(2018.08.03)
week = 8

DF_week = new_df[new_df['party_start_week'] == week]
DF_week_groupby = DF_week.groupby('hashed').agg('sum')
DF_week_groupby = DF_week_groupby.reset_index()

DF_week_groupby = DF_week_groupby.drop(['party_end_week', 'party_start_week'], axis = 1)
DF_week_groupby.columns = ['acc_id', str(week)+'_party_time']

DF_merge = pd.merge(left = DF_merge, right = DF_week_groupby, how='left', on='acc_id')

In [35]:
DF_merge.to_csv(path+'DF_train.csv', encoding=False, index=False)

## 3. id별로 파티 시간 합치기 - groupby<a name =3></a>

In [7]:
start = datetime.now()
new_df_2 = new_df.groupby('hashed').agg('sum')
new_df_2 = new_df_2.reset_index()
new_df_2.columns = ['acc_id', 'total_party_time']
print(datetime.now()-start)
print("Data shape : ",new_df_2.shape)
new_df_2.head()

0:01:06.555333
Data shape :  (268148, 2)


,acc_id,total_party_time
0,000020b4fa2af1e40c813436e5054bce70b703d0039dcd...,56786.555050
1,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,2514.369400
2,00006916672106f94f68d18e4a0532b53a40f4549d3a62...,33.826183
3,0000988c5778e9d6a861fee8fbfd88ec2fdc9108c1acd6...,620.352300
4,0000a2ff91f6891797708f346d8e3790fa75c1872d6fdc...,1011.070533


따로 설명이 필요 없다고 생각이 듭니다.

## 4. train_label 데이터와 병합 - merge<a name=4></a>

In [9]:
start = datetime.now()

DF_train = pd.merge(DF_label,new_df_2, how = 'left', on = 'acc_id')

print(datetime.now()-start)
print("Data shape : ",DF_train.shape)
DF_train.head()

0:00:00.205476
Data shape :  (100000, 3)


,acc_id,label,total_party_time
0,b8fbf3f6a70e3f36843bffc70c18ff51a0d755a87616ec...,week,696.442867
1,ed500c4957956b3e99dc3985666850b582f812405eefb6...,week,2244.687433
2,acc6afa23a6bf15e18151e4794c7789225ef9d682f473c...,week,334.495633
3,34095a3c9a2937ced3ea3fd75e22ce177dc5879d2a53f7...,week,NaN
4,26f3db6e8817a93c4ceda9a16f0832945e43d950b95882...,week,625.470767


0.24초 실화입니까??! 내가 짠 코드는 한 행에 34초. 총 34초 x 10,000행 = 340,000초 = 5,666분 = 94시간 = 3.9일이었는데 ....<br/>
간단하게 NaN이 몇 개 있고, NaN은 파티를 안 한 것을 의미하니 0으로 채워넣도록 하겠습니다.

In [10]:
DF_train.isnull().sum()

acc_id                  0
label                   0
total_party_time    45642
dtype: int64

In [11]:
DF_train = DF_train.fillna(0)
DF_train.isnull().sum()

acc_id              0
label               0
total_party_time    0
dtype: int64

4,5627개나 있네요 .. 혹시 코드가 잘못된 것은 아닌지 걱정이 됩니다 .... <br/>
토요일까지 남은 시간에 임의로 몇 개 뽑아서 확인 작업해야겠습니다.<br/>
마지막으로 csv파일로 저장하겠습니다.

In [12]:
DF_train.to_csv(path+'train.csv', index = False, encoding = False)

현 시점에서 걱정되는 2가지
1. 파티 시간의 분산이 너무 크다. 분 단위를 시간 단위로 바꿔야하는가?
2. 대충 데이터로 봤을 때, 큰 상관관계는 없어보인다.